# Relatório de Validação da Tabela STG_OPR_ITT

## Importando as bibliotecas e carregando os arquivos necessários

In [1]:
#Importa as bibliotecas necessárias
import pandas as pd
import numpy as np
import datetime
from plotly.offline import iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import os

In [2]:
#Cria as pastas necessárias
pasta ="./relatorio/pags"

access_rights = 0o777

try:
    os.mkdir('relatorio', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (relatorio) falhou ou a pasta já existe")
else:
    print ("\tPasta (relatorio) criada com sucesso!")

try:
    os.mkdir('relatorio/pags', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (pags) falhou ou a pasta já existe")
else:
    print ("\tPasta (pags) criada com sucesso!")

	AVISO: A criação da pasta (relatorio) falhou ou a pasta já existe
	AVISO: A criação da pasta (pags) falhou ou a pasta já existe


In [3]:
#Define as funções personalizadas necessárias
def write_to_html_file(df, title='', filename='out.html'):
    result = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    body{ 
        display: flex;
        align-items: center;
        justify-content: center;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
        width: 90%;
        color: #707070;
    }
    table, th, td {
        border: 1px solid #707070;
        border-collapse: collapse;
    }
    th {
        padding: 5px;
        text-align: left;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    td {
        padding: 5px;
        text-align: right;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }

</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    if type(df) == pd.io.formats.style.Styler:
        result += df.render()
    else:
        result += df.to_html(classes='wide', escape=False)
    result += '''
</body>
</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

def highlight_max(s):    
    is_max = s == s.max()
    return ['color: white; background-color: #3749E9' if v else '' for v in is_max]

def highlight_min(s):
    is_max = s == s.min()
    return ['color: white; background-color: #112244' if v else '' for v in is_max]

def destaque(val):
    color = '#F2F200' if val == 'Sim' else ''
    return 'background: {}'.format(color)

def destaque_coluna(val):
    color = '#F2F200' if val == 0 else ''
    return 'background: {}'.format(color)

In [4]:
# DataFrame principal.
dados_STG_OPR_ITT = pd.read_excel('STG_OPR_ITT.xlsx') # DataFrame de Operações.

# DataFrame para validação (esse será usado para validar as modalidades do principal).
dados_STG_MDL = pd.read_excel('STG_MDL.xlsx') # DataFrame das Modalidades.

# DataFrame para validação (esse será usado para validar os ID's das fontes do principal).
dados_STG_FNT_ITT = pd.read_excel('STG_FNT_ITT.xlsx') # DataFrame das Fontes.

In [5]:
#dados_STG_OPR_ITT

## Validando os tipos de dados das colunas

In [6]:
# Tratando as datas.
dados_STG_OPR_ITT.DAT_RSS_FNT_ITT = pd.to_datetime(dados_STG_OPR_ITT.DAT_RSS_FNT_ITT, format='%Y-%m-%d', errors='coerce')
dados_STG_OPR_ITT.DAT_INC_DBO = pd.to_datetime(dados_STG_OPR_ITT.DAT_INC_DBO, format='%Y-%m-%d', errors='coerce')

# Adicionando os tipos das respectivas colunas.
dict_tipo = {'ID_STG_OPR_ITT': int, 'VLR_CTRD_CSC': float, 'QTD_PCL': int, 'VLR_SDO_DDR': float,
             'QTD_CLI_CAD_POS': int, 'QTD_OPR': int, 'ID_FNT_ITT': int, 'ID_MDL': str,
             'DES_TIP_PSS': ('F' or 'J'), 'DAT_RSS_FNT_ITT': datetime.date, 'DAT_INC_DBO': datetime.date}

dict_analise = {}

# Realizando a verificação.
for coluna, tipo in dict_tipo.items():
    
    dict_analise[coluna] = len([linha for linha in dados_STG_OPR_ITT[coluna] 
                                if type(linha) == tipo or linha == tipo or isinstance(linha, tipo)])
            

total = dados_STG_OPR_ITT.shape[0]

# Convertendo o valor obtido para porcentagem.
final = [x * 100 / total for x in dict_analise.values()]

tipo_review = pd.DataFrame({'Porcentagem': final},
                           index = dict_analise.keys())
tipo_review = tipo_review.rename_axis('Colunas', axis = 'columns')
#tipo_review

In [7]:
#Plota em formato gráfico e exporta em HTML
data = [go.Bar(x = final,
               y = [x for x in dict_analise.keys()],
               orientation = 'h',
               marker=dict(color='#112244')
              )]

fig = go.Figure(data=data)
fig.update_yaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(xaxis=dict(ticksuffix = '%'))
iplot(fig)
fig.write_html(pasta + "/opr_tipo.html")

## Definindo dados sensíveis

In [8]:
#Cria um dataframe definido as colunas com dados sensíveis
dado_sensivel = {x: 'Não' for x in dados_STG_OPR_ITT.columns}
dado_sensivel['ID_FNT_ITT'] = 'Sim'

sensibilidade_review = pd.DataFrame({'Dado Sensível': dado_sensivel},
                                      index = dado_sensivel.keys())

sensibilidade_review = sensibilidade_review.rename_axis('Colunas', axis = 'columns')

visualiza = (sensibilidade_review
                .style
                .applymap(destaque))
write_to_html_file(visualiza, title='', filename=pasta + "/opr_tab_sensibilidade.html")
visualiza

## Validando as Modalidades e os ID's Fonte

### Modalidades

In [9]:
# Pegando os IDs para serem validado.
modalidades = dados_STG_OPR_ITT['ID_MDL']

# Atribuindo os códigos existentes em uma lista para poder usar o 'in'. 
codigos_existentes = [x for x in dados_STG_MDL['COD_MDL']] 

count_validos_mdl = len([codigo for codigo in modalidades if codigo in codigos_existentes])

porcentagem_mdl = count_validos_mdl * 100 / len(modalidades)

### ID Fonte

In [10]:
# Pegando os IDs para serem validado.
fontes = dados_STG_OPR_ITT['ID_FNT_ITT']

# Atribuindo os códigos existentes em uma lista para poder usar o 'in'. 
codigos_existentes = [x for x in dados_STG_FNT_ITT['ID_STG_FNT_ITT']] 

count_validos_fontes = len([codigo for codigo in fontes if codigo in codigos_existentes])

porcentagem_fontes = count_validos_fontes * 100 / len(fontes)

In [11]:
#Cria um dataframe
colunas_validar = ['Códigos de Modalidades', 'IDs de Fontes']
encontrados = [len(modalidades), len(fontes)]
porcentagens = [porcentagem_mdl, porcentagem_fontes]

correspondencia_review = pd.DataFrame({'Encontrados': encontrados,
                                       'Correspondência': porcentagens},
                                      index = colunas_validar)
correspondencia_review = correspondencia_review.rename_axis('Colunas', axis='columns')

visualiza = (correspondencia_review
                .style
                .format({'Correspondência':"{:.2f}%"})
                .applymap(destaque_coluna, subset=pd.IndexSlice[:, ['Correspondência']]))
write_to_html_file(visualiza, title='', filename=pasta + "/opr_tab_correspondencia.html")
visualiza

## Analisando Campos Únicos

In [12]:
#Analisa o número de valores únicos e que não foram duplicados
unico = [len(np.unique(dados_STG_OPR_ITT.ID_STG_OPR_ITT)) * 100 / len (dados_STG_OPR_ITT)]
ff = dados_STG_OPR_ITT[['VLR_CTRD_CSC', 'QTD_PCL', 'VLR_SDO_DDR',
         'QTD_CLI_CAD_POS', 'QTD_OPR', 'ID_FNT_ITT',
         'ID_MDL', 'DES_TIP_PSS',
         'DAT_RSS_FNT_ITT', 'DAT_INC_DBO']]
    
for x in ff.columns:
    coluna = dados_STG_OPR_ITT
    coluna = coluna.replace(0,np.nan)
    soma = coluna.groupby(f'{x}').count()
    soma = soma[soma.ID_STG_OPR_ITT == 1]
    soma = soma.ID_STG_OPR_ITT.sum()
    unico.append (soma * 100 / len (coluna[x]))

#Cria um dataframe
unico_review = pd.DataFrame({'Campos Não Duplicados': unico},
                           index=dados_STG_OPR_ITT.columns)
unico_review = unico_review.rename_axis('Colunas', axis='columns')
#unico_review

In [13]:
#Plota em gráfico de barras horizontal
data = go.Bar(y = unico, 
              x = dados_STG_OPR_ITT.columns, 
              marker = {'color' : '#00B7CC'})

layout = go.Layout(title = '', 
                   yaxis = {'title': ''}, 
                   xaxis = {'title': ''})

fig = go.Figure(data = data, layout = layout)
fig.update_xaxes(showline = True, linewidth = 1, linecolor = '#717171')
fig.update_yaxes(showgrid = True, gridwidth = 1, gridcolor = '#D9D9DE')
fig.update_layout({'plot_bgcolor': '#FFFFFF', 'paper_bgcolor': '#FFFFFF'})
fig.update_layout(yaxis=dict(ticksuffix = '%'))
iplot(fig)
fig.write_html(pasta + "/opr_unicos.html")

## Filtrando os casos especiais e analisando o preenchimento das células

In [14]:
# Completando espaços vázios com zeros.
dados_STG_OPR_ITT = dados_STG_OPR_ITT.fillna(0)

dict_tipo = {'ID_STG_OPR_ITT': np.int64, 'VLR_CTRD_CSC': np.float64, 'QTD_PCL': np.int64, 'VLR_SDO_DDR': np.float64,
             'QTD_CLI_CAD_POS': np.int64, 'QTD_OPR': np.int64, 'ID_FNT_ITT': np.int64, 'ID_MDL': str,
             'DES_TIP_PSS': str}

for coluna, tipo in dict_tipo.items():
    dados_STG_OPR_ITT[coluna] = dados_STG_OPR_ITT[coluna].astype(tipo)
    
#dados_STG_OPR_ITT.dtypes

## Analisando a Completude dos dados

In [15]:
#Completa espaços vazios com zero e exclui as linhas com valor zero
analisar = dados_STG_OPR_ITT.drop(columns = ['VLR_CTRD_CSC', 'VLR_SDO_DDR'])
analisar = analisar != 0
analisar = analisar.sum()

filtro = dados_STG_OPR_ITT[dados_STG_OPR_ITT.ID_MDL == 'C01']
consorcio = filtro[['VLR_CTRD_CSC', 'VLR_SDO_DDR']]
consorcio = consorcio != 0
consorcio = consorcio.sum()

situacao = [(x * 100) / len (dados_STG_OPR_ITT) for x in analisar]
situacao.extend ((x * 100) / len (filtro) for x in consorcio)

tabela_labels = [x for x in analisar.index]
tabela_labels.extend (x for x in consorcio.index)

#Cria um dataframe
tabela_review = pd.DataFrame({'COMPLETUDE (%)' : situacao},
                             index = tabela_labels)
tabela_review = tabela_review.rename_axis('Colunas', axis='columns')
tabela_review = tabela_review.T 
tabela_review = tabela_review[dados_STG_OPR_ITT.columns]
tabela_review = tabela_review.T 
#tabela_review

#### Criando um gráfico.

In [16]:
#Plota em formato gráfico e exporta para HTML
data = go.Bar(x = tabela_review['COMPLETUDE (%)'], 
              y = tabela_review.index, 
              orientation = 'h', 
              marker = {'color' : '#3749E9'})

layout = go.Layout(title = '', 
                   yaxis = {'title': ''}, 
                   xaxis = {'title': ''})

fig = go.Figure(data = data, layout = layout)
fig.update_yaxes(showline = True, linewidth = 1, linecolor = '#717171')
fig.update_xaxes(showgrid = True, gridwidth = 1, gridcolor = '#D9D9DE')
fig.update_layout({'plot_bgcolor': '#FFFFFF', 'paper_bgcolor': '#FFFFFF'})
fig.update_layout(xaxis=dict(ticksuffix = '%'))
iplot(fig)
fig.write_html(pasta + "/opr_completude_sem_consorcio.html")

## Recência dos Dados

In [17]:
#Analisando a recência de dados
analisar = dados_STG_OPR_ITT[['ID_STG_OPR_ITT', 'DAT_RSS_FNT_ITT']] 
DAT_RSS_FNT_ITT = analisar.groupby('DAT_RSS_FNT_ITT').count()

analisar = dados_STG_OPR_ITT[['ID_STG_OPR_ITT', 'DAT_INC_DBO']]
DAT_INC_DBO = analisar.groupby('DAT_INC_DBO').count()

colunas = [DAT_RSS_FNT_ITT, DAT_INC_DBO]
total_linhas = analisar.shape[0]
dados = []
ocorrencia = []
nome_coluna = []

for coluna in colunas:
    dados.extend(x for x in coluna.index.values)
    ocorrencia.extend([x * 100 / total_linhas for x in coluna.ID_STG_OPR_ITT.values])
    nome_coluna.extend([coluna.index.name] * (len(dados) - len(nome_coluna)))

analise =  pd.DataFrame({'Datas Encontradas' : dados, 
                         'Ocorrência' : ocorrencia}, 
                        index = nome_coluna)

analise = analise.rename_axis('Colunas', axis='columns')
analise.index.name = None

visualiza = (analise
                .style
                .format({'Ocorrência':"{:.2f}%"})
                .applymap(destaque_coluna, subset=pd.IndexSlice[:, ['Ocorrência']]))
write_to_html_file(visualiza, title='', filename=pasta + "/opr_tab_datas.html")
visualiza

# Indicadores de Validação

In [18]:
#Calcula a média de preenchimento da remessa
media_preenchimento = sum (situacao) / dados_STG_OPR_ITT.shape[1]

#Calcula de validação  
media_validacao = (sum (final) /dados_STG_OPR_ITT.shape[1] + sum (porcentagens) / len (porcentagens)  + sum (ocorrencia) / len (ocorrencia) ) / 3

#media_preenchimento, media_validacao